In [4]:
import io
import os.path

import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

In [14]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

def list_files():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes. 
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("drive", "v3", credentials=creds)

        # Call the Drive v3 API
        results = (
            service.files()
            .list(q=f"'{FEELINGS_LAB_FOLDER}' in parents", 
                  pageSize=10, 
                  includeItemsFromAllDrives=True, 
                  supportsAllDrives=True, 
                  fields="nextPageToken, files(id, name)")
            .execute()
        )
        items = results.get("files", [])

        if not items:
            print("No files found.")
            return
        print("Files:")
        for item in items:
            print(f"{item['name']} ({item['id']})")
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f"An error occurred: {error}")

list_files()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=179508738690-9tslh644kn4bfnpsribq85cip79tht86.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55600%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=cz3P7gEpsAnwEX6tOikd6Kli1M21h2&access_type=offline
Files:
Emotion Analysis (17pvsEMrayBuP_lBx9btpmRDlC6yQVXQw)
HumeAI (15Ost8rQZyIs-3JfpzlHNEsb0wEfompUU)
Animated Video (1ywmzFb2AgUNhGxz15x2QHsh_lcGGWQvv)
Season 2 (1rOsMW2dWv5-Am3L5Sb941rKd-ITi1SQj)
Season 1 (1lpRblj3vMS-_kqaGKtF7B6pjQgN5emeN)
Creative Assets  (1WBTX-9BawC9pzeYK5kppk6mcIe0G3sdP)


In [29]:
def traverse_folders(service, folder_id, path, target_folders):
    """Recursively traverse folders to find .wav files and download them.
    Args:
        service: Authorized Drive API service instance.
        folder_id: ID of the folder to traverse.
        path: Current path of traversal.
        target_folders: List of target folders to traverse.
    """
    results = (
        service.files()
        .list(q=f"'{folder_id}' in parents", 
              pageSize=1000, 
              includeItemsFromAllDrives=True, 
              supportsAllDrives=True, 
              fields="nextPageToken, files(id, name, mimeType)")
        .execute()
    )
    items = results.get("files", [])

    for item in items:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            if path == '' or any(folder in path for folder in target_folders):
                print(f"Traversing {item['name']} at {path}")
                traverse_folders(service, 
                                 item['id'], path + '/' + item['name'], 
                                 target_folders)
        elif item['name'].endswith('.wav') and any(folder in path for folder in target_folders):
            file_path = 'audio/' + item['name']
            if not os.path.exists(file_path):
                print(f"Downloading {item['name']} from {path}")
                download_file(service, item['id'], file_path)
            else: 
                print(f"File {item['name']} already exists at {file_path}")

def download_file(service, file_id, file_path):
    """Downloads a file
    Args:
        service: Authorized Drive API service instance.
        file_id: ID of the file to download.
        file_path: Path to save the downloaded file.
    """
    request = service.files().get_media(fileId=file_id)
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}%.")

    # Save the file to the specified path
    with open(file_path, 'wb') as f:
        f.write(file.getvalue())

In [31]:
FEELINGS_LAB_FOLDER = "11fFbCqntrywSWNE7B0TDwj6XwjoDhFrb"
FOLDERS_TO_TRAVERSE = ['Season 1', 'Season 2']
creds = Credentials.from_authorized_user_file("token.json", SCOPES)
service = build("drive", "v3", credentials=creds)
traverse_folders(service, FEELINGS_LAB_FOLDER, '', FOLDERS_TO_TRAVERSE)

File S2_Ep008_Master.wav already exists at audio/S2_Ep008_Master.wav
File TFL-S2_Ep005_Master.wav already exists at audio/TFL-S2_Ep005_Master.wav
File TFL_S2_EP004_Master.wav already exists at audio/TFL_S2_EP004_Master.wav
File TFL_S2_EP003_Master_AUDIO.wav already exists at audio/TFL_S2_EP003_Master_AUDIO.wav
File TFL_S2E1_Final_Audio.wav already exists at audio/TFL_S2E1_Final_Audio.wav
File TFL_S2_Ep011_Master.wav already exists at audio/TFL_S2_Ep011_Master.wav
File TFL_Ep10_MASTER_Final.wav already exists at audio/TFL_Ep10_MASTER_Final.wav
File TFL_Ep09_Master_AUDIO.wav already exists at audio/TFL_Ep09_Master_AUDIO.wav
File TFL_Ep08_Final.wav already exists at audio/TFL_Ep08_Final.wav
File TFL_Ep07_MasterAUDIO.wav already exists at audio/TFL_Ep07_MasterAUDIO.wav
File TFL_Episode06_Master.wav already exists at audio/TFL_Episode06_Master.wav
File TFL_Episode05_FINAL.wav already exists at audio/TFL_Episode05_FINAL.wav
File TFL_ep05_HORROR_THEME.wav already exists at audio/TFL_ep05_HORR